In [ ]:
from layer_diffuse.data_loaders import ModularCharatersDataLoader
from layer_diffuse.models import DDIMNextTokenV1_Refactored
import json
pipeline = DDIMNextTokenV1_Refactored.DDIMNextTokenV1PipelineRefactored()
vocab_file = "layer_diffuse/vocab.json"
with open(vocab_file, 'r') as f:
    vocab = json.load(f)
dataloader = ModularCharatersDataLoader.get_modular_char_dataloader(dataset_name='QLeca/modular_characters_v3',
                                                                            split='train',
                                                                            image_size=128,
                                                                            batch_size=8,
                                                                            shuffle=True,
                                                                            streaming=True,
                                                                            conversionRGBA=True,
                                                                            vocab=vocab)

In [ ]:
pipeline.list_versions()

In [ ]:
pipeline.load_model_from_hub(run='run_2025-06-23_19-12-13',
                             epoch=49)
pipeline.set_num_class_embeds(len(dataloader.vocab))

In [ ]:
import torch
import torchvision
from torchvision.utils import make_grid

def show_image_grid(input_images, output_images, target_images):
    output_images = (output_images * 0.5 + 0.5).clamp(0, 1).cpu()
    input_images = (input_images * 0.5 + 0.5).clamp(0, 1).cpu()
    target_images = (target_images * 0.5  + 0.5).clamp(0, 1).cpu()
    concat = torch.concat([input_images, output_images, target_images])
    grid = make_grid(concat, nrow=input_images.shape[0])
    img = torchvision.transforms.ToPILImage()(grid)
    display(img)


In [ ]:
for batch in dataloader:
    input_images = batch['input']
    target_images = batch['target']
    labels = batch['label']
    outputs = pipeline(input_images=input_images, 
                       class_labels=labels,
                       num_inference_steps=50)
    show_image_grid(input_images, outputs, target_images)
    break    